<a href="https://colab.research.google.com/github/zhejindong/DATABASE_DEVELEPMENT/blob/master/Project2_s19_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 2: Graph Analysis
* Released: 04/16
* Due: 05/02 10AM
* Value: 5% of your grade
* Max team of 2

Many graph analysis compute network centrality, density, shortest paths, and other path-based statistics about a graph.  It may seem that writing a one-off Python script is a good way to perform this analysis, but it turns out that SQL is pretty great at doing this type of analysis!  

## Background of the Data
For this assignment, you will do a graph analysis to analyze Tweets from [**Twitter Elections Integrity Dataset**](https://about.twitter.com/en_us/values/elections-integrity.html#data).

- Justice Department charged 13 Russian nationals with interfering in American electoral and political processes. The defendants worked for a well-funded “troll factory” called the Internet Research Agency(**IRA**).
- IRA ran a campaign to sow disinformation and discord into American politics via social media (mostly twitter).
- Dataset includes information from 3613 accounts believed to be connected to the Russian Internet Research Agency.

#### Twitter IRA dataset

In reality, the twitter dataset contains the following attributes.

```
tweetid                   # tweet id
userid                    # user id (hashed for users which had fewer than 5,000 followers) 
user_display_name         # name of user (same as userid for anonymized users)
user_screen_name          # the Twitter handle of the user
user_reported_location    # self-reported location
……
```

Please check [Twitter Elections Integrity Datasets Readme](https://storage.googleapis.com/twitter-election-integrity/hashed/Twitter_Elections_Integrity_Datasets_hashed_README.txt) for full description

## Refresher

You will write queries or short Python programs to answer the following questions about the dataset.  

In the simple case, graphs have the following schema:

        nodes(id int primary key, <attributes>)
        edges(
          src int NOT NULL references nodes(id),
          dst int NOT NULL references nodes(id),
          <attributes
        )

Recall that in graph analysis, you are interested in finding neighbors of nodes or paths between nodes.    Following an edge in the graph corresponds to a JOIN.  For example, the following finds all neighbors of node id 2:

        SELECT dst FROM edges WHERE src = 2;

Similarly, if we have a table `goodnodes` that contains IDs of nodes that we are interested in, the following query finds neighbors of these good nodes:

        SELECT dst FROM edges, goodnodes WHERE edges.src = goodnodes.id;

## Before you begin


### Step 1. Setup BigQuery
* Click **File** on the colab menu, chose **"Save a copy in Drive"**, so you can save your work to your Google Drive.
* Follow [this instruction](https://github.com/w4111/project2_s19/blob/master/Setup_Instructions.pdf) to enable BigQuery, get project ID and create dataset to store your temperary table

### Step 2. Important BigQuery Reference

BigQuery support standard SQL functions and operators. If you need help with SQL syntax, the following reference will be really helpful.

[**BigQuery Standard SQL Functions & Operators API Reference**](https://cloud.google.com/bigquery/docs/reference/standard-sql/functions-and-operators)

### Step 3. Provide your and your teammate UNI

In [190]:
# Your columbia uni that is used in SSOL
My_UNI = "zd2221"
My_Teammate_UNI = "None"

print("My UNI is {u1}, My teammate UNI is {u2}".format(u1=My_UNI, u2 = My_Teammate_UNI))

My UNI is zd2221, My teammate UNI is None


### Step 4. Provide your credentials to the runtime
Please click on the link in the output of following cell, get the token, type it in and hit Enter.

In [191]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


### Step 5. Test your settings and check the IRA dataset

Please fill in your Google Project ID below and run the cell.

* `%%bigquery --project your_project_id` is a magic command to run BigQuery in Colab. To get `your_project_id`, please follow the instructions in **step 1**.

* `project2-236400.twitter.IRA`  is the address of IRA Dataset.

In [0]:
# set the max display column
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [0]:
%%bigquery --project w4111-project-2-239513
SELECT *
FROM `project2-236400.twitter.IRA` 
LIMIT 5

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,...,latitude,longitude,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions,poll_choices
0,719916264425185280,201334945,Андрей Манзолевский,Manzal_,Russia,"Блогер, публицист, гражданин, начальник",http://t.co/lsUS2Frfjj,23245,3298,2010-10-11,...,,,0,1,0,0,[],[],[116529466],
1,750310622089314304,201334945,Андрей Манзолевский,Manzal_,Russia,"Блогер, публицист, гражданин, начальник",http://t.co/lsUS2Frfjj,23245,3298,2010-10-11,...,,,0,0,1,0,[],[],"[3178891317, 92821589]",
2,151699476556414976,201334945,Андрей Манзолевский,Manzal_,Russia,"Блогер, публицист, гражданин, начальник",http://t.co/lsUS2Frfjj,23245,3298,2010-10-11,...,,,0,0,0,0,,,,
3,515038385757368320,201334945,Андрей Манзолевский,Manzal_,Russia,"Блогер, публицист, гражданин, начальник",http://t.co/lsUS2Frfjj,23245,3298,2010-10-11,...,,,0,0,0,0,,[http://j.mp/1u0Q2Oq],,
4,262522018640056320,201334945,Андрей Манзолевский,Manzal_,Russia,"Блогер, публицист, гражданин, начальник",http://t.co/lsUS2Frfjj,23245,3298,2010-10-11,...,,,0,0,0,0,,,,


## Question 1

Find the `id` and `text` of Tweets that contain  "MakeAmericaGreatAgain" **and** "Trump" (both **case-insensitive**) .  

For example:

* "#VoteTrump and lets all help #**Trump** #**makeamericagreatagain**" is a match
* "#**Trump** This was our moment. Together, we will **make America great again**!" is not a match

Your anwser should be a single query. You need to run it and get the output containing the columns:
- id (tweetid of the tweets)
- text (tweet_text of the tweets)

###Write your SQL below

In [0]:
%%bigquery --project w4111-project-2-239513
SELECT tweetid, tweet_text 
FROM `project2-236400.twitter.IRA` 
WHERE tweet_text  like "%MakeAmericaGreatAgain%" or 
      tweet_text  like "%Trump%"

,tweetid,tweet_text
0,759390739046563840,#IHatePokemonGoBecause this is pro Trump
1,725860620805926913,#SanJose Protests Expected at Donald Trump's Bay Area Event
2,752938883004850177,"#politics Sanders endorses Clinton, pledges to help defeat Trump"
3,740966934955462657,House Republicans seek to shift focus from Trump to security https://t.co/EdvammRPyk
4,707473377720606720,The Latest: Donald Trump wins Hawaii Republican caucuses #politics
5,681669833407074304,Trump attacks publisher of largest New Hampshire newspaper #politics
6,692266474707275776,#politics How Rubio plans to take on Trump: Attrition
7,625727514258747392,#politics Disgruntled GOP voters provide the Trump bump
8,785185781031718912,A fact-checker guide to sex charges that Donald Trump might raise https://t.co/FXMyYbpIFK
9,655039303777521664,#politics Trump: 9/11 happened on Bush's watch


## Question 2
"**MakeAmericaGreatAgain**" (often abbreviated as MAGA) is a campaign slogan used in American politics that was popularized by Donald Trump in his successful 2016 presidential campaign.

Let's Find out Top 5 month gets the maximum mention of this slogan
"MakeAmericaGreatAgain"(**case-insensitive**) in `tweet_text` .

Your anwser should be a single query. You need to run it and get the output containing the columns:
- year
- month
- num (number of tweets mentioned the slogan in the month)

###Write your SQL below

In [0]:
%%bigquery --project w4111-project-2-239513
SELECT
  EXTRACT(YEAR FROM tweet_time) AS year,
  EXTRACT(MONTH FROM tweet_time) AS month,
  count(*) as num
FROM `project2-236400.twitter.IRA`
WHERE tweet_text  like "%MAGA%" or 
      tweet_text  like "%MakeAmericaGreatAgain%"
group by year, month 
order by num desc
limit 5


,year,month,num
0,2017,8,6587
1,2017,3,6352
2,2017,4,5888
3,2017,5,2286
4,2017,6,1999


## Question 3   
**Let's construct the Graph!**

Tweets and retweets can be used to construct the graph where each row is an edge between `userid` and `retweet_userid`. Create a table “Graph” with column names src and dst which stores the edge list of the graph. You must store only the distinct edges in the table. One user might retweet another user's tweet more than one time. In this case, you should only save the edge once (i.e. only one row in the graph table).

This question doesn't need output. You must save this table since you will be using it for the next few questions. Your table should contain the following columns:
- src (**retweet_userid**)
- dst (**userid**)



---


To save a table, issue a regular query using :

``CREATE OR REPLACE TABLE dataset_name.table_name AS SELECT ...``

For example, 
    ```
    CREATE OR REPLACE TABLE `4111dataset.tmp` AS
    SELECT *
    FROM `project2-236400.twitter.IRA` 
    LIMIT 5
    ```


###Write your SQL below

In [192]:
%%bigquery --project w4111-project-2-239513
CREATE OR REPLACE TABLE dataset.Graph as SELECT distinct retweet_userid as src, userid as dst from `project2-236400.twitter.IRA` where is_retweet=True

""


## Question 4
The indegree of a node in a directed graph is defined as the number of edges which are incoming on the node. Similarly, the outdegree of a node in a directed graph is defined as the number of edges which are outgoing from the node. For more information, you can read - [Indegree and Outdegree](https://en.wikipedia.org/wiki/Directed_graph#Indegree_and_outdegree)

Using this information, find out from the GRAPH table which user has the highest indegree and which user has the highest outdegree.

Your anwser should be a single query. You need to run it and get the output containing the columns:
- max_indegree (userid)
- max_outdegree (userid)

###Write your SQL below

In [0]:
%%bigquery --project w4111-project-2-239513
WITH ind as (select dst, count(*) as indegree from  dataset.Graph group by dst),
     oud as (select src, count(*) as outdegree from  dataset.Graph group by src),
     temp1 as(select dst as max_indegree from ind where indegree=(SELECT max(indegree) as max_indegree FROM ind )),
     temp2 as(select src as max_outdegree from oud where outdegree=(SELECT max(outdegree) as max_outdegree FROM oud ))
     select max_indegree, max_outdegree from temp1, temp2
     

,max_indegree,max_outdegree
0,a95a911dd6ae864c48ed062cdbe75e5c28dbe0cf57c6db3fa9dd53b654ee9845,2572896396


## Question 5
Let us define 4 categories of Twitter users in IRA dataset. We will use the average number of likes a user gets on his/her tweets as the first metric and the number of users retweet him (i.e. indegree) as the second metric. Then we can classify each user as follows:
- High indegree, high average number of likes 
- High indegree, low average number of likes
- Low indegree, high average number of likes
- Low indegree, low average number of likes

We will refer to the 'low indegree, low average likes' category of users as "unpopular" users and 'high indegree, high average likes' category of users as "popular" users. 

We define the indegree and average number of likes to be high or low based on the rules below:
   
   1) If `indegree < avg(indegree of all users)` in the graph then indegree is said to be low for the user, else it is considered high.
    
   2) If `avg(likes of all tweets for the user) < avg(likes for all tweets)` in the graph, then the average number of likes is said to be low for the user, else it is considered high.

Now, you need to find the conditional probability that given **all tweets from unpopular users**, what is the probability that **they are retweeted from popular users**. We only consider user in this dataset.

You can use temperary tables to do this question(no need to implement in a single query). Your final output should contain the column:
- popular_unpopular (conditional probability)

###Write your SQL below

In [0]:
%%bigquery --project w4111-project-2-239513
WITH ind as (select dst, count(*) as indegree from  dataset.Graph group by dst),
     pop_in as (select distinct dst from ind where indegree >(select avg(indegree) from ind)),
     unpop_in as (select distinct dst from ind where indegree < (select avg(indegree) from ind)),
     like_table as (select userid,avg(like_count) as like_num from `project2-236400.twitter.IRA` group by userid),
     unpop_like as (select distinct userid from like_table where like_num<(select avg(like_count) from `project2-236400.twitter.IRA`)),
     pop_like as (select distinct userid from like_table where like_num>(select avg(like_count) from `project2-236400.twitter.IRA`)),
     unpop_user as (select * from unpop_in INTERSECT DISTINCT select * from unpop_like),
     pop_user as (select * from pop_in INTERSECT DISTINCT select * from pop_like),
     unpop_pop as(select count(*) as num from dataset.Graph where dst in (select * from pop_user) and src in (select * from unpop_user)),
     unpop as (select count(*) as num from dataset.Graph where src in (select * from unpop_user))
     select unpop_pop.num/unpop.num*1.0 as condition_proba
     from unpop, unpop_pop 
     

,condition_proba
0,0.00252


## Question 6
Given a graph G = (V, E), a “triangle” is a set of three vertices that are mutually adjacent in G i.e. given 3 nodes of a graph A, B, C there exist edges A->B, B->C and C->A which form a triangle in the graph. From the graph table which you created above, find out the number of triangles in the graph.

In a directed graph, the direction of the edges matter.
 **A -> B -> C -> A** and **A -> C -> B -> A** will count as 2 triangles.

Your anwser should be a single query. You need to run it and get the output containing the columns:
- no_of_triangles



###Write your SQL below

In [186]:
%%bigquery --project w4111-project-2-239513
with temp1 as (
select d1.src as src1, d1.dst as dst1, d2.src as src2, d2.dst as dst2
from
dataset.Graph as d1,dataset.Graph as d2
where 
d1.src=d2.dst),
temp2 as (
select distinct src1,dst1,src2,dst2  from temp1, dataset.Graph as d3 
where temp1.dst1=d3.src and temp1.src2=d3.dst)
select CAST(count(*)/3 AS INT64) as no_of_triangles from temp2 


,no_of_triangles
0,1519382


## Question 7
Let's check who has the most impact in IRA dataset.

The **PageRank algorithm** is used to rank the importance of nodes in a graph. It works by counting the number of edges incident to a node to determine how important the node is. The underlying assumption is that more important nodes are likely to receive more links from other nodes. Find the top 100 nodes with the highest PageRank in the graph.
Hint: It is not possible to use "WITH RECURSIVE" on BigQuery. You must develop a iterative implementation for PageRank (like the BFS example mentioned below).

You must run the algorithm for 20 iterations and your output table should contain the following columns:
- twitter_username (the twitter_username of the user)
- page_rank_score



---


This algorithm works as follows - Assume a small universe of four web pages: A, B, C and D. PageRank is initialized to the same value for all pages since we assume a probability distribution between 0 and 1 as the PageRank for each node. Hence the initial value for each page in this example is 0.25. If the only links in the system were from pages B->A, C->A and D->A, each link would transfer 0.25 PageRank to A upon the next iteration, for a total of 0.75 i.e. PR(A) = PR(B) + PR(C) + PR(D). 

Now, suppose instead that we have the links B->C, B->A, C->A, D->A, D->B, D->C. Thus, upon the first iteration, page B would transfer half of its existing value, or 0.125, to page A and the other half, or 0.125, to page C. Page C would transfer all of its existing value, 0.25, to the only page it links to, A. Since D had three outbound links, it would transfer one third of its existing value, or approximately 0.083, to A. At the completion of this iteration, page A will have a PageRank of approximately 0.458.
PR(A)=PR(B)/2 + PR(C)/1 + PR(D)/3.

Thus, we can write the PageRank of A as:
PR(A)= PR(B)/L(B) + PR(C)/L(C) + PR(D)/L(D) where L(x) gives us the number of outbound links for any node x. 

In general, the PageRank value for a page u is dependent on the PageRank values for each page v contained in the set containing all pages linking to page u, divided by the number of links from page v. 
It is given by the formula: ![](https://www.geeksforgeeks.org/wp-content/ql-cache/quicklatex.com-aafd3a0d9f8bb8325cf2b41a4a839bbf_l3.svg)



---



#### For this question, you can implement the simplified version of the PageRank algorithm. But it will be great if you handle dangling node and reducible graph. To read more about PageRank, you can refer to the following link: [PageRank](http://home.ie.cuhk.edu.hk/~wkshum/papers/pagerank.pdf)

you will need to develop an iterative solution, i.e. your python code will act as a driver and issue multiple queries to BigQuery iteratively. As an example, we provided an iterative implementation of Breadth First Search on the starter code.

To execute 5 iterations using A as a start node, you can simply call ``bfs(client, 'A', 5)``.

The example saves the nodes visited at each iteration in a table ``distances``, along with their distance to the initial node. The function itself does not return any value (however, remind that you will be required to return values for Q7).

### Do not edit this function. This is for helping you develop your own iterative PageRank algorithm.

You don't have to use the BFS function in Q7. 
 It is just an example to show you how to write iterative functions using SQL.  

In [0]:
project_id = 'w4111-project-2-239513'

In [181]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

def bfs(client, start, n_iter):

    # You should replace dataset.bfs_graph with your dataset name and table name.
    q1 = """
        CREATE TABLE IF NOT EXISTS dataset.bfs_graph (src string, dst string);
        """
    q2 = """
        INSERT INTO dataset.bfs_graph(src, dst) VALUES
        ('A', 'B'),
        ('A', 'E'),
        ('B', 'C'),
        ('C', 'D'),
        ('E', 'F'),
        ('F', 'D'),
        ('A', 'F'),
        ('B', 'E'),
        ('B', 'F'),
        ('A', 'G'),
        ('B', 'G'),
        ('F', 'G'),
        ('H', 'A'),
        ('G', 'H'),
        ('H', 'C'),
        ('H', 'D'),
        ('E', 'H'),
        ('F', 'H');
        """

    job = client.query(q1)
    results = job.result()
    job = client.query(q2)
    results = job.result()

    # You should replace dataset.distances with your dataset name and table name. 
    q3 = """
        CREATE OR REPLACE TABLE dataset.distances AS
        SELECT '{start}' as node, 0 as distance
        """.format(start=start)
    job = client.query(q3)
    # Result will be empty, but calling makes the code wait for the query to complete
    job.result()

    for i in range(n_iter):
        print("Step %d..." % (i+1))
        q1 = """
        INSERT INTO dataset.distances(node, distance)
        SELECT distinct dst, {next_distance}
        FROM dataset.bfs_graph
            WHERE src IN (
                SELECT node
                FROM dataset.distances
                WHERE distance = {curr_distance}
                )
            AND dst NOT IN (
                SELECT node
                FROM dataset.distances
                )
            """.format(
                curr_distance=i,
                next_distance=i+1
            )
        job = client.query(q1)
        results = job.result()
       

In [187]:
bfs(client, 'A', 5)

Step 1...
Step 2...
Step 3...
Step 4...
Step 5...


###Write your Python Code below

In [194]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)
def PageRank(client, n_iter):
  q1="""
  with t1 as (
  SELECT distinct src from dataset.Graph

  union distinct 

  select distinct dst from dataset.Graph)

  select count(*) as total_node from t1
  """
  job = client.query(q1)
  results = job.result() 
  total_node=results.to_dataframe()
  total=total_node['total_node'].values[0]
  
  q4 = """
        CREATE OR REPLACE TABLE dataset.page_rank AS
        SELECT distinct node, 1.0/{total} as pagerank
        FROM 
        (
        SELECT distinct src as node from dataset.Graph

        union distinct 

        select distinct dst as node from dataset.Graph
        )     
       """.format(total=total)
  job = client.query(q4)
  results = job.result() 

  # create table outdegree in  clout 
  q5="""
  CREATE OR REPLACE TABLE dataset.Out as (select src, count(*) as outdegree from  dataset.Graph group by src)
  """
  job = client.query(q5)
  results=job.result()
  # update the parameter 
  
  for i in range(n_iter):
    print("Step %d..." % (i+1))
    q6="""
    create or replace table dataset.give as 
    (select node as node,pagerank/outdegree as give
    from dataset.page_rank as page_rank, dataset.Out as Out 
    where page_rank.node=Out.src)
    """
    job = client.query(q6)
    results=job.result()

    q7="""
    CREATE OR REPLACE TABLE dataset.page_rank AS(
    with temp1 as 
    (select * 
    from dataset.give as give,dataset.Graph as Graph 
    where give.node=Graph.src
    ),
     temp2 as 
    (select dst, sum(give) as gain from temp1 group by dst),

    temp3 as(
    select * 
    from dataset.page_rank as page_rank,temp2 
    where page_rank.node=temp2.dst)

    select node,gain as pagerank
    from temp3)
    """
    job = client.query(q7)
    results = job.result()

In [195]:
PageRank(client, 20)

Step 1...
Step 2...
Step 3...
Step 4...
Step 5...
Step 6...
Step 7...
Step 8...
Step 9...
Step 10...
Step 11...
Step 12...
Step 13...
Step 14...
Step 15...
Step 16...
Step 17...
Step 18...
Step 19...
Step 20...


In [197]:
## select the top 10 pages 

%%bigquery --project w4111-project-2-239513
select * from dataset.page_rank
order by pagerank desc
limit 10


,node,pagerank
0,a95a911dd6ae864c48ed062cdbe75e5c28dbe0cf57c6db3fa9dd53b654ee9845,0.081719
1,ce88fa855f9d0064cd85cd7938b4b484b4de4966d2462802a40e72b98ce354b0,0.005243
2,432b5b8c124aa79f278df08b1961f474c14b2f0aaee2091b18d02667d08ebea0,0.005239
3,f8fb4be6969cb8dfd5bba5f58174ae52f46125e0109ff5bd1bae7f5c1d277746,0.005239
4,52097ebf36ea47af5e4e8a3c8c4f36076bc31345c1e58b73e1407f8da4523dab,0.005239
5,f3f40074ccd82cc468b182cdf6772e5aa609a39b3b7f7c49ce497c782bd470a9,0.005235
6,7da03b08bdc22426ad8550af049eda83f3546e4cef3f023c59d719495f0e9135,0.005235
7,271484d1d8f811c7184958d7d25b346f3733d2dc1d265eb8bd75764f0991b3b1,0.005108
8,544ffc2a3ab3cf83eccf320cc33921c8afa5e4e8864973e0410563235555c4ba,0.005108
9,fe2395083be2ad32c75843525d6ab75e9ae8b69ba94d585cca04b0592ee121b4,0.005105


## Submission Instructions
* Click **File** on the colab menu, chose **Download .ipynb**

* Submit it via:
https://www.instabase.com/apps/file-submission/cmd/submit/05dd2e62-11b6-4219-9e59-2ede5ebc642b
